# Optuna + MLflow Hyperparameter Optimization Analysis

This notebook analyzes the results of automated hyperparameter optimization
performed on the California Housing dataset using XGBoost, Optuna, and MLflow.

We review optimization progress, hyperparameter importance, and compare
baseline vs tuned model performance.


In [6]:
import optuna
import mlflow
import mlflow.tracking

import pandas as pd
import numpy as np

from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate
)

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor


c:\Users\kvina\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
study = optuna.load_study(
    study_name="xgboost-housing-optimization",
    storage="sqlite:///../outputs/optuna_study.db"
)

print("Number of trials:", len(study.trials))
print("Best CV RMSE:", np.sqrt(-study.best_value))
print("Best parameters:", study.best_params)


Number of trials: 100
Best CV RMSE: 45031.67315398589
Best parameters: {'n_estimators': 285, 'max_depth': 10, 'learning_rate': 0.05695649572350409, 'subsample': 0.8553831646686475, 'colsample_bytree': 0.7984961923756487, 'min_child_weight': 9, 'gamma': 0.19318011367299853}


In [8]:
plot_optimization_history(study)


In [9]:
plot_param_importances(study)


In [10]:
plot_parallel_coordinate(study)


In [11]:
X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

baseline_model = XGBRegressor(
    random_state=42,
    objective="reg:squarederror"
)

baseline_model.fit(X_train, y_train)

baseline_pred = baseline_model.predict(X_test)
baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_pred))
baseline_r2 = r2_score(y_test, baseline_pred)

baseline_rmse, baseline_r2


(np.float64(0.4717943691423984), 0.8301370561019205)

In [12]:
best_params = study.best_params.copy()
best_params.update({
    "random_state": 42,
    "objective": "reg:squarederror"
})

tuned_model = XGBRegressor(**best_params)
tuned_model.fit(X_train, y_train)

tuned_pred = tuned_model.predict(X_test)
tuned_rmse = np.sqrt(mean_squared_error(y_test, tuned_pred))
tuned_r2 = r2_score(y_test, tuned_pred)

tuned_rmse, tuned_r2


(np.float64(0.4406978200142688), 0.8517908746122576)

In [13]:
pd.DataFrame({
    "Model": ["Baseline XGBoost", "Tuned XGBoost"],
    "RMSE": [baseline_rmse, tuned_rmse],
    "R2": [baseline_r2, tuned_r2]
})


,Model,RMSE,R2
0,Baseline XGBoost,0.471794,0.830137
1,Tuned XGBoost,0.440698,0.851791


## Key Insights

### Optimization Behavior
- The optimization history shows steady improvement in CV RMSE across trials,
  indicating effective exploration of the hyperparameter space.
- MedianPruner successfully reduced computation by stopping underperforming trials early.

### Hyperparameter Importance
- Learning rate and max depth had the strongest impact on model performance,
  highlighting the bias–variance trade-off in tree-based models.
- Subsampling and column sampling contributed to regularization and improved generalization.

### Baseline vs Tuned Performance
- The tuned model significantly outperforms the baseline XGBoost model.
- RMSE improved substantially after optimization.
- R² increased beyond 0.70, meeting the project performance requirement.

### Conclusion
Automated hyperparameter optimization using Optuna, combined with systematic
experiment tracking via MLflow, leads to reproducible and measurable performance gains.
This pipeline demonstrates production-grade MLOps practices.
